# <center>Construction de la Base</center>

## Import des Librairies

In [2]:
import csv
import datetime
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

## Connexion à la Base et Création du Curseur

In [10]:
# Create a connection to the database
conn = psycopg2.connect(dbname="Twitter-Elections", user="postgres", host="localhost", port="5432", password="password")

# Create a cursor to perform database operations
cur = conn.cursor()

# Create an SQLAlchemy engine just to use Pandas read_sql_queries
engine = create_engine("postgresql://postgres@localhost:5432/Twitter-Elections")

## Création des Tables

In [24]:
# Delete all the tables from the database if there are any
cur.execute("""
    DROP SCHEMA public CASCADE;
    CREATE SCHEMA public;
    GRANT ALL ON SCHEMA public TO postgres;
    GRANT ALL ON SCHEMA public TO public;
""")

In [25]:
# Create the tables from the SQL script
with open("db_creation.sql", "r") as f:
    cur.execute(f.read())

In [26]:
# Check if tables were correctly created
cur.execute("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema='public'
        AND table_type='BASE TABLE';
   """)

cur.fetchall()

[('candidats',),
 ('tweets',),
 ('affiliation_elections',),
 ('contexte_elections',),
 ('denomination_partis',),
 ('tweets_tags',),
 ('tags',)]

## Insertion des Données Externes

### Table `candidats`

In [27]:
# Delete all entries from candidats if there are already any
cur.execute("DELETE FROM candidats")

# Reset the SERIAL primary key to make it start at 1
cur.execute("ALTER SEQUENCE candidats_id_seq RESTART")

In [28]:
# Insert data from a csv file
with open("data/candidats.csv", newline="") as f:
    reader = csv.reader(f, delimiter=";")
    headers = next(reader, None)
    for row in reader:
        row = [item if item!='' else None for item in row ]
        cur.execute("""
            INSERT INTO candidats (prenom, nom, sexe, date_naissance, id_twitter, username, compte_verifie, date_creation_compte) 
            VALUES (%s, %s, %s, TO_DATE(%s, 'DD/MM/YYYY'), %s, %s, %s, TO_DATE(%s, 'DD/MM/YYYY'))
            """, row)
        
conn.commit()

In [29]:
candidats = pd.read_sql_query("""
        SELECT 
            id,
            prenom,
            nom,
            sexe,
            date_naissance,
            CAST(id_twitter AS VARCHAR),
            username,
            compte_verifie,
            date_creation_compte
        FROM candidats
        """,con=engine, coerce_float=False)

candidats

,prenom,nom,sexe,date_naissance,id_twitter,username,compte_verifie,date_creation_compte
id,,,,,,,,
1,Sandrine,Adobati,F,1974-03-30,None,None,None,None
2,Marie-Josee,Boulaire,F,1959-01-26,1130169982917180000,boulairejosee,False,2019-05-01
3,Arnaud,Charvillat,M,1977-10-28,None,None,None,None
4,François-Marie,Didier,M,1980-10-03,1207401907129659397,fmdidier,False,2019-12-01
5,Lamia,El Aaraje,F,1986-11-22,943672818,lamiaela,True,2012-11-01
6,Mohamad-Lamine,Gassama,M,None,376865423,MohamadParis20,False,2011-11-01
7,Philippe,Houplain,M,1961-07-16,3255832605,HouplainPh,False,2015-05-01
8,Danielle,Simonnet,F,1971-07-02,329686497,Simonnet2,False,2011-07-01
9,Apolline,Tyburczy,F,1987-10-12,1489995966132412422,ApollineTyburzy,False,2022-02-01


### Table `denomination_partis`

In [30]:
# Delete all entries from denomination_partis if there are already any
cur.execute("DELETE FROM denomination_partis")

In [31]:
# Insert data from a csv file
with open("data/denomination_partis.csv", newline="") as f:
    reader = csv.reader(f, delimiter=";")
    headers = next(reader, None)
    for row in reader:
        row = [item if item!='' else None for item in row ]
        cur.execute("""
            INSERT INTO denomination_partis
                (nom_annee_election, nom_derniere_election) 
            VALUES (%s, %s)
            """, row)
        
conn.commit()

In [32]:
denomination_partis = pd.read_sql_query("""
        SELECT *
        FROM denomination_partis
        """,con=engine, coerce_float=False)

denomination_partis

,nom_annee_election,nom_derniere_election
0,EXG,DXG
1,DXG,DXG
2,FG,COM
3,COM,COM
4,FI,FI
5,SOC,SOC
6,RDG,RDG
7,DVG,DVG
8,VEC,VEC
9,REG,REG


### Table `contexte_elections`

In [33]:
# Delete all entries from contexte_elections if there are already any
cur.execute("DELETE FROM contexte_elections")

In [34]:
# Insert data from a csv file
with open("data/contexte_elections.csv", newline="") as f:
    reader = csv.reader(f, delimiter=";")
    headers = next(reader, None)
    for row in reader:
        row = [item if item!='' else None for item in row ]
        cur.execute("""
            INSERT INTO contexte_elections (annee, parti_vainqueur_presidentielles) 
            VALUES (%s, %s)
            """, row)
        
conn.commit()

In [35]:
contexte_elections = pd.read_sql_query("""
        SELECT *
        FROM contexte_elections
        """,con=engine, coerce_float=False)

contexte_elections

,annee,parti_vainqueur_presidentielles
0,2012,PS
1,2017,LREM
2,2022,LREM


### Table `affiliation_elections`

In [36]:
# Delete all entries from affiliation_elections if there are already any
cur.execute("DELETE FROM affiliation_elections")

# Reset the SERIAL primary key to make it start at 1
cur.execute("ALTER SEQUENCE affiliation_elections_id_seq RESTART")

In [37]:
# Insert data from a csv file
with open("data/affiliation_elections.csv", newline="") as f:
    reader = csv.reader(f, delimiter=";")
    headers = next(reader, None)
    for row in reader:
        row = [item if item!='' else None for item in row ]
        cur.execute("""
            INSERT INTO affiliation_elections (
                id_candidat, 
                annee_election,
                nom_annee_election,
                code_departement,
                code_circonscription,
                sortant,
                dissident,
                resultat_election) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, row)
        
conn.commit()

In [39]:
contexte_elections = pd.read_sql_query("""
        SELECT *
        FROM affiliation_elections
        """,con=engine, coerce_float=False)

contexte_elections

,id,id_candidat,annee_election,nom_annee_election,code_departement,code_circonscription,sortant,dissident,resultat_election
0,1,1,2022,DIV,75,075-15,False,False,None
1,2,2,2022,RN,75,075-15,False,False,None
2,3,3,2022,EXG,75,075-15,False,False,None
3,4,4,2022,LR,75,075-15,False,False,None
4,5,5,2022,SOC,75,075-15,True,True,None
5,6,6,2022,REM,75,075-15,False,False,None
6,7,7,2022,REC,75,075-15,False,False,None
7,8,8,2022,FI,75,075-15,False,False,None
8,9,9,2022,ECO,75,075-15,False,False,None
9,10,10,2017,FN,75,075-15,False,False,False


## Close the Connection and Cursor

In [16]:
# Closing of of cursor and connection
cur.close()
conn.close()